# Brain-Machine Interface Spike Sorting with PCA

### EE 16B: Designing Information Devices and Systems II, Summer 2020

## Table of Contents

* [Introduction](#introduction)
* [Task 1: Two Neuron Spike Sorting](#task1)
* [Task 2: Three Neuron Spike Sorting](#task2)
* [Task 3: Determining Neurons](#task3)

<a id='introduction'></a>
# Introduction



For SIXT33N (your robot car), you will need to use SVD and PCA to classify your sound input (either words or music). This exercise will help make sure you are prepared for the next two labs. Make sure you complete this problem before your lab.

Brain-Machine interfaces (BMIs) are a way for a brain to communicate to an external device. They're often used for research, mapping, assisting or repairing human cognitive or sensory-motor function. 

Data was collected that shows spike timestamps of simulated brain waves of a subject whose arm is pointing in certain directions over time. These spike timestamps are gathered from electrodes, but the electrodes could be recording more than 1 neuron at the same time. To make predictions based on neuron firing rates, we need to be able to distinguish spikes that come from different neurons. 

Luckily for us, it is highly unlikely for two neurons next to each other to fire at precisely the same time, so in practice the "average" potential waveform looks like a sum of spike waveforms from different neurons. Additionally, each neuron has its own spike "signature" unique to that neuron. This is due to the physical characteristics of neurons, such as their shape and structure. It is impossible to know beforehand how many neurons an electrode measures or what each neuron's spike waveform looks like, so we must first separate the spikes from the different neurons near the electrode.

The goal of this problem is to see how we can use PCA and clustering to decide which neuron fired. We will first look at a case where there are only two neurons, then a case where there are three neurons. We provide both training and test sets with two and three neurons. The neurons have also been presorted using a professional software, so we can check our model against presorted data.


## Task 1: Two Neuron Spike Sorting

A spike occurs when the potential passes a certain threshold in a neuron. In our data set, each recorded waveform contains `N=32` samples taken at a sampling frequency of 40kHz, which means the time difference between each consecutive sample is 800$\mu$s. We can represent each vector exactly with any 32-dimensional basis, as long as the vectors making up the basis are linearly independent. However, we want to compute a minimal set of basis vectors that brings out the features of the data using Principal Component Analysis (PCA).

Import the data sets and see the average waveform for each presorted neuron by running the following cell.

In [ ]:
%matplotlib inline
import numpy as np
import scipy.io
import scipy.cluster
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load data
presorted = {k: v for k, v in scipy.io.loadmat('spike_waveforms').items() \
             if k in ('sig118a_wf', 'sig118b_wf', 'sig118c_wf')}
presorted = [presorted['sig118a_wf'], presorted['sig118b_wf'], presorted['sig118c_wf']]

In [ ]:
def _make_training_set(data):
    """ Separate data set into 2 sets. 
    1/6 of the dataset is training set and the rest is test set
    Parameter:
        data: waveform data (width = number of samples per spike)
    """
    n = data.shape[0]
    idx_training = np.random.choice(n, n//6, replace=False)
    training_set = data[idx_training]
    test_set = [data[i] for i in range(n) if n not in idx_training]
    return training_set, test_set

# Create training and testing dataset
two_neurons_training, two_neurons_test = _make_training_set(np.concatenate(presorted[1:]))
three_neurons_training, three_neurons_test = _make_training_set(np.concatenate(presorted))

Let's first see what the spikes look like. Below, we first plot 100 random spikes. Then, we plot the 3 distinct neuron's shape, taking the average over all of the samples gathered from that neuron based on the presorted data. The goal of this lab is to classify each of the line in the first plot to one of the neurons in the second plot.

In [ ]:
# Plot 100 random spikes
for waveforms in three_neurons_training[:100]:
    plt.plot(waveforms)
plt.xlim((0,31))
plt.title('100 random spikes')

# Plot the 3 spike shapes based on the presorted data
plt.figure()
for waveforms in presorted:
    plt.plot(np.mean(waveforms, axis=0))
plt.xlim((0,31))
plt.title('Averaged presorted 3 neuron spikes')

We can represent each spike in a lower dimensional space using PCA. In your implementation of PCA you will decide how to choose these components.

**<font color="red">Using PCA, what will be the principal components if we use data from an electrode that measures 3 neurons?** Describe a special property of the first two principal components (Hint: how does it relate to the third principal component?).

YOUR ANSWER HERE

You will be using <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html">np.linalg.svd</a> in your PCA function. Read the documentation for this function to figure out how to choose the principal components used as the basis for the lower dimensional space. (Note: in the docs, `a.H` means the congugate transpose of a).

**<font color="red">What does SVD return and how will you use those results?**

YOUR ANSWER HERE

Complete the functions below to implement PCA training and classifying. Don't forget that you will need to offset the data by the mean!

In [ ]:
def PCA_train(training_set, n_components):
    """ Use np.linalg.svd to perform PCA
    Parameters:
        training_set: the data set to perform PCA on (MxN)
        n_components: the dimensionality of the basis to return (i.e. number of neurons)
    Returns: 
        The n_components principal components with highest significants and 
        the mean of each column of the original data
    Hint: Subtract the mean of the data first so the average of each column is 0. The axis 
        parameter of np.mean is helpful here.
    """    
    # YOUR CODE HERE #
    return basis_components, mean

def PCA_classify(data, new_basis, mean):
    """ Project the data set, adjusted by the mean, into the new basis vectors
    Parameters:
        data: data to project (MxN)
        new_basis: new bases (KxN)
        mean: mean of each timestamp from PCA (list of length N)
    Returns: 
        Data projected onto new_basis (MxK)
    Hint: Don't forget to adjust the data with the PCA training mean!
    """
    # YOUR CODE HERE #

Nowl let's call `PCA_train` on `two_neurons_training` and plot the 2 principal components. Note that since the dataset is randomized, you might get different plots every time you run the second code cell of this notebook (the `_make_training_set` function). Note that these components have no real 'physical' relationship to the shape of the neuron plots.

In [ ]:
# Perform PCA and plot the first 2 principal components.

# YOUR CODE HERE

# Plot the basis components
for comp in two_new_basis:
    plt.plot(comp)

Now, classify `two_neurons_test` using that model and produce a scatter plot in the new basis. We will also try classifying the presorted data containing 2 neurons so we can see how the model behaves on the 2 neurons.

In [ ]:
# Project the test data two_neurons_test to the basis you found earlier

# YOUR CODE HERE #


# Plot the classified neurons
plt.figure()
plt.scatter(*two_classified.T)
plt.title('two_neurons_test')

# Project the presorted data and plot it
plt.figure()
presorted_two_classified = [PCA_classify(spikes, two_new_basis, two_mean) for spikes in presorted[1:]]
colors = ['#0000ff', '#00ff00']
for dat, color in zip(presorted_two_classified, colors):
    plt.scatter(*dat.T, c=color)
plt.title('Presorted data')

Note that the first principal component separates the two neurons in the $x$-axis. Thus, technically we only need 1 principal component to separate the two neurons. This is because the algorithm maximizes the square of the dot product of each signal with the principal component, which results in a large positive dot product with 1 neuron and a large negative dot product with the other.

## Task 2: Three Neuron Spike Sorting

Now call `PCA_train` on `three_neurons_training` and plot the 3 principal components.

In [ ]:
# Repeat training with three neuron data, producing 3 principal components

# YOUR CODE HERE


# Plot the resulting basis
for comp in three_new_basis:
    plt.plot(comp)

Do the same classification process as the 2 neuron data, but now with the 3 neuron data. Compare your model's behavoir with that of the presorted data. The `plot_3D` function below will be useful to view the results.

In [ ]:
def plot_3D(data, view_from_top=False):
    """ Takes list of arrays (x, y, z) coordinate triples
    One array of triples per color
    """
    fig=plt.figure(figsize=(10,7))
    ax = fig.add_subplot(111, projection='3d')
    colors = ['#0000ff', '#00ff00', '#ff0000']
    for dat, color in zip(data, colors):
        Axes3D.scatter(ax, *dat.T, c=color)
    if view_from_top:
        ax.view_init(elev=90.,azim=0)                # Move perspective to view from top

        
# Classify the three_neurons_test data based on the basis computed above
# YOUR CODE HERE #



# Plot the resulting classificaiton
plot_3D([three_classified], False)
plt.title('three_neurons_test projected to 3 principal components')

presorted_classified = [PCA_classify(spikes, three_new_basis, three_mean) for spikes in presorted]
plot_3D(np.array(presorted_classified), False)
plt.title('Presorted data projected to 3 principal components')

**<font color="red">How many principal components do you actually need to cluster the 3 neurons?** Change the second argument to the `plot_3D` function calls above to True to view the plots "from the top" (i.e. looking down the positive z axis).

YOUR ANSWER HERE

## Task 3: Determining Neurons

Now that we have seperated the data into clusters, we can use k-means clustering to determine how many times each neuron fired. We can use this information to create a rough estimate for the average firing rate of each neuron over the time spanned by the given data.

Scipy just happens to have a function to compute k-means given clustered data, which we have conveniently formatted formated above. First, read the documentation for the function scipy.cluster.vq.kmeans <a href="http://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.kmeans.html#scipy.cluster.vq.kmeans">here</a>.

**<font color="red">Based on the documentation for scipy.cluster.vq.kmeans, how will we be able to adjust our approach for the 2-neuron and 3-neuron data sets?**

YOUR ANSWER HERE

Now that we are familiar with how the function works, lets try to determine how many times each neuron fired in the 2-neuron data set. Start by determining the centroids of each cluster.

In [ ]:
def find_centroids(clustered_data, num_of_clusters):
    """ Use scipy.cluster.vq.kmeans to determine centroids of clusters
    Parameters:
        clustered_data: the data projected onto the new basis
        num_of_clusters: the expected number of clusters in the data
    Returns: 
        The centroids of the clusters
    Hint 1: make sure to first 'whiten' the data (refer to docs)
    """
    whitened_data = scipy.cluster.vq.whiten(clustered_data)
    return scipy.cluster.vq.kmeans(whitened_data, num_of_clusters)[0]

Now, call find_centroids on the projected 2-neuron data.

In [ ]:
# Determine the centroids in the 2-neuron data

# YOUR CODE HERE #


# Print the centroid locations
centroid1 = centroid_list[0]
centroid2 = centroid_list[1]

print('The first centroid is at: ' + str(centroid1))
print('The second centroid is at: ' + str(centroid2))

One thing to note about the centroids we just found is that they have been normalized (by the "whiten" process), so they don't appear to match with the plot we made above. However, they should match the general location of the two clusters.

Now that we have the location of the clusters, we can determine which neuron fired for each data point. We will do this by determing the centroid closer to the data point in question

In [ ]:
def which_neuron(data_point, centroid1, centroid2):
    """ Determine which centroid is closest to the data point
    Inputs:
        data_point: 1x2 array containing x/y coordinates of data point
        centroid1: 1x2 array containing x/y coordinates of centroid 1
        centroid2: 1x2 array containing x/y coordinates of centroid 1
    Returns: 
        The centroid closest to the data point
    """
    
    # YOUR CODE HERE

Now apply this to the two_classified data set and count the number of times each neuron fired

In [ ]:
# Determine how many times neuron1 and neuron2 fired in the two_classifed data
# Hint: remember that we "whitened" the data to determine the centroids

num_of_firings = np.zeros(2)

# YOUR CODE HERE
    
# Print the results
print('Neuron 1 Fired ' + str(num_of_firings[0]) + ' times')
print('Neuron 2 Fired ' + str(num_of_firings[1]) + ' times')